In [6]:
import geopandas as gpd
import pandas as pd
import datetime

In [2]:
import sys
sys.path.append('..')

In [3]:
import rsutils.s2_grid_utils
import create_datacube

In [4]:
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'

In [5]:
east_bb_gdf = gpd.read_file(roi_filepath).to_crs('epsg:4326')

In [ ]:
create_datacube.get_intersecting_band_filepaths(
    shapes_gdf = east_bb_gdf,
    catalog_filepath = '../data/satellite/Sentinel-2/catalog.geojson',
    startdate = datetime.datetime(2021, 3, 1),
    enddate = datetime.datetime(2022, 1, 31),
    bands = ['B08']
)

In [ ]:
east_bb_gdf

In [9]:
east_bb_s2grids_gdf = rsutils.s2_grid_utils.get_s2_grids_gdf(geojson_epsg_4326=east_bb_gdf['geometry'][0])

In [15]:
pd.DataFrame(
    east_bb_s2grids_gdf[['id']].rename(columns={'id':'s2gridid'})
).to_csv('../data/ethiopia_east_bb_s2grids.csv', index=False)

In [16]:
datacube_catalog_gdf = gpd.read_file('../data/cluster_files/catalog.geojson')

In [19]:
east_bb_s2grids_gdf[
    ~east_bb_s2grids_gdf['id'].isin(datacube_catalog_gdf['roi_name'])
][['id']].rename(columns={'id':'s2gridid'}).to_csv('../data/failed_ethiopia_east_bb_s2grids.csv', index=False)